# 言語モデル

*Language Model*

単語の並びに確率を割り当てるモデル。  
文章生成に多く使われるので、文章生成モデルと見てもいいが、当然別の用途もある。

最近では、かなり人間に近い文章を生成できるすげーモデルが*LLM: Large Language Model*と称して一世を風靡している。

In [1]:
import random
from typing import List


---

## 言語モデルを用いた文章生成

任意の言語モデルを用いて文章を生成する方法を見ていく。

答えは簡単。確率が高い単語の並びを選択するだけ。  
モデルは語彙を持っており、その語彙から作れる全ての単語列に対して確率を割り当て、最も高い単語列を出力する。この単語列が、モデルが生成した文章となる。

実際に文章を作ってみる。

まずは言語モデルを作る。ここでは、いかなる単語列=文章に対してランダムな確率を出力するモデルを作る。

In [2]:
# 文章を入力として、確率を出力する関数
def langage_model(sentence):
    prob = random.random() # 0 ~ 1 のランダムな値
    return prob

できた。試しに文章を入れてみる。

In [3]:
sentence = "明日は晴れるといいな。"
prob = langage_model(sentence)
prob

0.7017995591740749

確率が出てきた。これを使えば文章が生成できそうだ。

では作っていこう、といきたいところだが、実は先程述べた生成手法には色々と問題がある。

先程、モデルが持っている語彙から作れる全ての単語列に確率を割り当てると説明した。しかし、語彙の数が大きい場合、作ることのできる単語列は膨大な量になってしまう。

そこで、**与えられた文脈**と持っている語彙から作れる単語列で考える。

例えば、モデルが以下の語彙を持っているとする。

- 今日
- 天気
- おいしい
- は
- いい

ここで、「今日 は いい」という単語列があったとする。このとき、この単語列と語彙から作れる文章を以下とする。

- 今日 は いい 今日
- 今日 は いい 天気
- 今日 は いい おいしい
- 今日 は いい は
- 今日 は いい いい

単語列の末尾に単語を1つ追加しただけだ。

言語モデルは、これらに対して確率を割り当て、最も確率の高い単語列を出力する。そして、出力された単語列を再度モデルに与え、同様に単語列を生成する。これを繰り返すことで、文章を生成する。このような繰り返しの処理を**再帰的**な処理と呼ぶ。

このような言語モデルの挙動は、与えられた文章から次の単語を予測しているように見える。言語モデルは「次の単語の予測」を繰り返すことで文章を生成する。モデルに与える文章は**文脈**と呼ぶ。

もう少し厳密な解釈をすると、言語モデルは、与えられた文脈に続く単語の**確率分布**を予測するモデルとなる。その確率分布に従ってサンプリングを行うことで次の単語を出力し、それを繰り返すことで文章を生成する。

では、この手法に基づいた言語モデルを作っていこう。

与えられた文脈から次の単語の確率分布を予測し、それに基づいて次の単語をサンプリングするモデルを作る。確率分布はランダムとする。

In [4]:
class LanguageModel:
    def __init__(self, vocab: List[str]):
        self.vocab = vocab # 語彙

    def prob_predict(self, sentence): # 単語列=文章を入力として、確率を出力する
        prob = random.random()
        return prob

    def sampling(self, dist): # 確率分布に従って単語をサンプリングする
        word, = random.choices(self.vocab, weights=dist)
        return word

    def __call__(self, context):
        # 各単語の確率を求める
        probs = []
        for word in self.vocab:
            sentence = context + word # 文脈+単語で文章を生成
            prob = self.prob_predict(sentence) # 文章を入力して確率を得る
            probs.append(prob)

        dist = [prob / sum(probs) for prob in probs] # 正規化
        word = self.sampling(dist) # 単語をサンプリング
        return word


できた。

適当に語彙と文脈を与えて次の単語を出力させてみよう。

In [5]:
vocab = ["今日", "天気", "おいしい", "は", "いい"] # 語彙
model = LanguageModel(vocab) # 言語モデル

context = "今日はいい" # 文脈
word = model(context) # 次の単語をサンプリング
word

'は'

この単語を文脈に追加して、再度次の単語を出力させる。

In [6]:
context = "今日はいい" + word # 得られた単語を文脈に追加
print(context)
word = model(context) # 次の単語をサンプリング
word

今日はいいは


'いい'

これを繰り返せば文章が出来そうだ。やってみよう。

In [7]:
n_words = 5 # 生成する単語数
sentence = "" # 最初の文脈

for _ in range(n_words):
    word = model(sentence) # 次の単語を予測
    sentence += word # 文脈に単語を追加

sentence

'はいい天気はは'

出来た。これが言語モデルによる文章生成である。

今回は、いかなる単語列に対してランダムな確率を出力するモデルを作り、それを用いて文章を生成した。出力される確率はランダムなので、当然、めちゃくちゃな文章が生成される。

ただ、もしモデルが、我々が自然だと感じる単語の並びに高い確率を割り当てられるようになった場合はどうだろう。我々にとって自然な文章が生成できるようになるのではないだろうか。

次章以降では、人間が書いた大量の文章を用いて（学習させて）、モデルが自然な文章を生成できるようにする。自然な単語列に高い確率を割り当てられるようにする。